## Big Data Day 2 Afternoon Assignment

In this assignment, we will learn about machine learning with Dask. We will use the market basket dataset loaded below and cluster our data.

In [1]:
# !pip install distributed>=2.4.0 --quiet

# !pip install dask-ml --quiet
# !pip install -U ipykernel --quiet

In [2]:
import dask.dataframe as dd
import numpy as np
import pandas as pd

In [3]:
market_basket = dd.read_csv('../data/Mall_Customers.csv')

In [4]:
market_basket.head()

,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


To cluster our data, we must first examine and process it. The first step is to check for missing data. Do this in the cell below. If there is missing data, drop all rows containing missing data.

In [5]:
# Answer below:
market_basket.compute().isna().mean()

CustomerID                0.0
Gender                    0.0
Age                       0.0
Annual Income (k$)        0.0
Spending Score (1-100)    0.0
dtype: float64

Next, we will get rid of the customer ID column. Drop the column and assign the resulting dataframe to a new variable.

In [6]:
# Answer below:
market_new = market_basket.drop(columns='CustomerID')
market_new.compute()

,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,Male,19,15,39
1,Male,21,15,81
2,Female,20,16,6
3,Female,23,16,77
4,Female,31,17,40
...,...,...,...,...
195,Female,35,120,79
196,Female,45,126,28
197,Male,32,126,74
198,Male,32,137,18


We will now create a dummy variable from the gender variable.

In [7]:
# Answer below:
market_dum = pd.get_dummies(market_new['Gender'])
market_dum

,Female,Male
0,0,1
1,0,1
2,1,0
3,1,0
4,1,0
...,...,...
195,1,0
196,1,0
197,0,1
198,0,1


In [8]:
market_new = pd.concat([market_new.compute(), market_dum], axis=1)

In [9]:
market_new = market_new.drop(columns='Gender')
market_new

,Age,Annual Income (k$),Spending Score (1-100),Female,Male
0,19,15,39,0,1
1,21,15,81,0,1
2,20,16,6,1,0
3,23,16,77,1,0
4,31,17,40,1,0
...,...,...,...,...,...
195,35,120,79,1,0
196,45,126,28,1,0
197,32,126,74,0,1
198,32,137,18,0,1


Since the data has different scales, we will scale all columns using min max scaling. We will write our own min max scaling function since the minmax scaler will produce numpy arrays instead of dask arrays. Recall that min max scaling requires finding the min and the max. We subtract the min from each observation and divide by the difference between the max and the min. Complete the function below.

In [10]:
# Answer below:
from sklearn.preprocessing import MinMaxScaler

def dask_min_max(df):
    #Input: dask column
    
    #Output: dask column that has been min max scaled
    scaler = MinMaxScaler()
    df_scaled = scaler.fit_transform(df)
    
    return df_scaled


Transform all columns using the scaler you wrote above.

In [11]:
# Answer below:
market_scaled = dask_min_max(market_new)
market_scaled

array([[0.01923077, 0.        , 0.3877551 , 0.        , 1.        ],
       [0.05769231, 0.        , 0.81632653, 0.        , 1.        ],
       [0.03846154, 0.00819672, 0.05102041, 1.        , 0.        ],
       [0.09615385, 0.00819672, 0.7755102 , 1.        , 0.        ],
       [0.25      , 0.01639344, 0.39795918, 1.        , 0.        ],
       [0.07692308, 0.01639344, 0.76530612, 1.        , 0.        ],
       [0.32692308, 0.02459016, 0.05102041, 1.        , 0.        ],
       [0.09615385, 0.02459016, 0.94897959, 1.        , 0.        ],
       [0.88461538, 0.03278689, 0.02040816, 0.        , 1.        ],
       [0.23076923, 0.03278689, 0.7244898 , 1.        , 0.        ],
       [0.94230769, 0.03278689, 0.13265306, 0.        , 1.        ],
       [0.32692308, 0.03278689, 1.        , 1.        , 0.        ],
       [0.76923077, 0.04098361, 0.14285714, 1.        , 0.        ],
       [0.11538462, 0.04098361, 0.7755102 , 1.        , 0.        ],
       [0.36538462, 0.04098361, 0.

Import the dask clustering function and cluster the data. Use 3 clusters and print the centroids.

In [12]:
# Answer below

